In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions
import org.apache.spark.sql.functions._
import com.databricks.spark.xml
import spark.implicits._

val sk = SparkSession.builder().appName("Parsing XML").master("local").getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://172.29.42.126:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1645130622294)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions
import org.apache.spark.sql.functions._
import com.databricks.spark.xml
import spark.implicits._
sk: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@55ca9fa3


In [2]:
val origDf = sk.read.format("com.databricks.spark.xml").option("rootTag","measData ").option("rowTag","measInfo").load("/home/saluka/dmstasks/network.xml").
        drop("granPeriod").drop("repPeriod")

origDf.show()
origDf.printSchema()

+-------------------+--------------------+--------------------+
|        _measInfoId|           measTypes|           measValue|
+-------------------+--------------------+--------------------+
|           LTE_S1AP|M8000C11 M8000C6 ...|{PLMN-PLMN/MRBTS-...|
|      LTE_Cell_Load|M8001C0 M8001C1 M...|{PLMN-PLMN/MRBTS-...|
| LTE_Transport_Load|M8004C0 M8004C2 M...|{PLMN-PLMN/MRBTS-...|
|LTE_Pwr_and_Qual_UL|M8005C345 M8005C5...|{PLMN-PLMN/MRBTS-...|
|     LTE_EPS_Bearer|M8006C181 M8006C1...|{PLMN-PLMN/MRBTS-...|
|   LTE_Radio_Bearer|M8007C0 M8007C1 M...|{PLMN-PLMN/MRBTS-...|
|            LTE_RRC|M8008C1 M8008C3 M...|{PLMN-PLMN/MRBTS-...|
|   LTE_Intra_eNB_HO|M8009C0 M8009C1 M...|{PLMN-PLMN/MRBTS-...|
|LTE_Pwr_and_Qual_DL|M8010C36 M8010C37...|{PLMN-PLMN/MRBTS-...|
|  LTE_Cell_Resource|M8011C86 M8011C95...|{PLMN-PLMN/MRBTS-...|
|LTE_Cell_Throughput|M8012C19 M8012C20...|{PLMN-PLMN/MRBTS-...|
|       LTE_UE_State|M8013C5 M8013C13 ...|{PLMN-PLMN/MRBTS-...|
|   LTE_Inter_eNB_HO|M8014C0 M8014C6 M..

origDf: org.apache.spark.sql.DataFrame = [_measInfoId: string, measTypes: string ... 1 more field]


In [3]:
// val df2 = origDf.select("*","measValue.*") 
val df2 = origDf.select(col("*"), col("measValue.measResults").as("measResults")).drop("measValue")
df2.show()

+-------------------+--------------------+--------------------+
|        _measInfoId|           measTypes|         measResults|
+-------------------+--------------------+--------------------+
|           LTE_S1AP|M8000C11 M8000C6 ...|20284 0 0 0 0 0 0...|
|      LTE_Cell_Load|M8001C0 M8001C1 M...|3 294 52 0 0 7172...|
| LTE_Transport_Load|M8004C0 M8004C2 M...|60537 788296 1132...|
|LTE_Pwr_and_Qual_UL|M8005C345 M8005C5...|-90 1 1 4 2 2 7 1...|
|     LTE_EPS_Bearer|M8006C181 M8006C1...|6 23 2 2 0 10 0 0...|
|   LTE_Radio_Bearer|M8007C0 M8007C1 M...|38 38 1 1 1 1 32 ...|
|            LTE_RRC|M8008C1 M8008C3 M...|20288 16269 2 2 2...|
|   LTE_Intra_eNB_HO|M8009C0 M8009C1 M...|1 15 0 0 0 0 0 0 ...|
|LTE_Pwr_and_Qual_DL|M8010C36 M8010C37...|26 17 114 377 255...|
|  LTE_Cell_Resource|M8011C86 M8011C95...|900 900 900 900 1...|
|LTE_Cell_Throughput|M8012C19 M8012C20...|188793 5858527 31...|
|       LTE_UE_State|M8013C5 M8013C13 ...|1 3 1 1 1 1 1 1 3...|
|   LTE_Inter_eNB_HO|M8014C0 M8014C6 M..

df2: org.apache.spark.sql.DataFrame = [_measInfoId: string, measTypes: string ... 1 more field]


In [12]:
val df3 = df2.select(col("*"), functions.split(df2("measTypes"), " ").as("Meas_Types"), functions.split(df2("measResults"), " ").as("Meas_Results")).
            withColumn("Types&Results", arrays_zip(col("Meas_Types"), col("Meas_Results"))).
            drop("measTypes").drop("measResults").withColumnRenamed("_measInfoId", "Meas_Info_Id")
df3.show()
df3.printSchema()
df3.count()

+-------------------+--------------------+--------------------+--------------------+
|       Meas_Info_Id|          Meas_Types|        Meas_Results|       Types&Results|
+-------------------+--------------------+--------------------+--------------------+
|           LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|[{M8000C11, 20284...|
|      LTE_Cell_Load|[M8001C0, M8001C1...|[3, 294, 52, 0, 0...|[{M8001C0, 3}, {M...|
| LTE_Transport_Load|[M8004C0, M8004C2...|[60537, 788296, 1...|[{M8004C0, 60537}...|
|LTE_Pwr_and_Qual_UL|[M8005C345, M8005...|[-90, 1, 1, 4, 2,...|[{M8005C345, -90}...|
|     LTE_EPS_Bearer|[M8006C181, M8006...|[6, 23, 2, 2, 0, ...|[{M8006C181, 6}, ...|
|   LTE_Radio_Bearer|[M8007C0, M8007C1...|[38, 38, 1, 1, 1,...|[{M8007C0, 38}, {...|
|            LTE_RRC|[M8008C1, M8008C3...|[20288, 16269, 2,...|[{M8008C1, 20288}...|
|   LTE_Intra_eNB_HO|[M8009C0, M8009C1...|[1, 15, 0, 0, 0, ...|[{M8009C0, 1}, {M...|
|LTE_Pwr_and_Qual_DL|[M8010C36, M8010C...|[26, 17, 114, 377...|[{

df3: org.apache.spark.sql.DataFrame = [Meas_Info_Id: string, Meas_Types: array<string> ... 2 more fields]
res10: Long = 157


In [15]:
//val df4 = df3.withColumn("Meas_Types",explode(col("Meas_Types"))).withColumn("Meas_Results",explode(col("Meas_Results")))
val df4 = df3.withColumn("Types&Results",explode(col("Types&Results")))
df4.show()
df4.printSchema()
df4.count()

+------------+--------------------+--------------------+-----------------+
|Meas_Info_Id|          Meas_Types|        Meas_Results|    Types&Results|
+------------+--------------------+--------------------+-----------------+
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|{M8000C11, 20284}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|     {M8000C6, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|     {M8000C7, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|     {M8000C8, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|     {M8000C9, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|    {M8000C13, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|    {M8000C14, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|    {M8000C15, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|    {M8000C16, 0}|
|    LTE_S1AP|[M8000C11, M8000C...|[20284, 0, 0, 0, ...|    {M8000C24, 0}|
|    LTE_S1AP|[M8000C11, 

df4: org.apache.spark.sql.DataFrame = [Meas_Info_Id: string, Meas_Types: array<string> ... 2 more fields]
res13: Long = 4292


In [16]:
val df5 = df4.select(col("Meas_Info_Id"),col("Types&Results.Meas_Types"), col("Types&Results.Meas_Results"))
df5.show()
df5.printSchema()
df5.count()

+------------+----------+------------+
|Meas_Info_Id|Meas_Types|Meas_Results|
+------------+----------+------------+
|    LTE_S1AP|  M8000C11|       20284|
|    LTE_S1AP|   M8000C6|           0|
|    LTE_S1AP|   M8000C7|           0|
|    LTE_S1AP|   M8000C8|           0|
|    LTE_S1AP|   M8000C9|           0|
|    LTE_S1AP|  M8000C13|           0|
|    LTE_S1AP|  M8000C14|           0|
|    LTE_S1AP|  M8000C15|           0|
|    LTE_S1AP|  M8000C16|           0|
|    LTE_S1AP|  M8000C24|           0|
|    LTE_S1AP|  M8000C26|           0|
|    LTE_S1AP|  M8000C27|           0|
|    LTE_S1AP|  M8000C29|           0|
|    LTE_S1AP|  M8000C30|           0|
|    LTE_S1AP|  M8000C32|           0|
|    LTE_S1AP|  M8000C33|           0|
|    LTE_S1AP|  M8000C34|           0|
|    LTE_S1AP|  M8000C35|           0|
|    LTE_S1AP|  M8000C36|           0|
|    LTE_S1AP|  M8000C37|           0|
+------------+----------+------------+
only showing top 20 rows

root
 |-- Meas_Info_Id: string (nullab

df5: org.apache.spark.sql.DataFrame = [Meas_Info_Id: string, Meas_Types: string ... 1 more field]
res14: Long = 4292


Save the dataframe to DB

In [6]:
import org.apache.spark.sql.SaveMode;

//JDBC initialization
val dbUrl = "jdbc:postgresql://localhost:5432/DMS"
import java.util.Properties
val connectionProperties = new Properties()

connectionProperties.put("user", s"postgres")
connectionProperties.put("password", "postgres")
connectionProperties.setProperty("Driver", "org.postgresql.Driver")

//Rename df columns and write into a postgres table
df5.write
  .mode(SaveMode.Overwrite)
  .jdbc(dbUrl, "Nokia_Networks", connectionProperties)

println("Saved to database successfully !")

Saved to database successfully !


import org.apache.spark.sql.SaveMode
dbUrl: String = jdbc:postgresql://localhost:5432/DMS
import java.util.Properties
connectionProperties: java.util.Properties = {password=postgres, Driver=org.postgresql.Driver, user=postgres}


In [7]:
//Read table contents to verify
val qry = "(select * from Nokia_Networks) as nokia"
val nkDf = spark.read.jdbc(dbUrl, qry, connectionProperties)
nkDf.show()

+------------+----------+------------+
|Meas_Info_Id|Meas_Types|Meas_Results|
+------------+----------+------------+
|    LTE_S1AP|  M8000C11|       20284|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
|    LTE_S1AP|  M8000C11|           0|
+------------+----------+------------+
only showing top 20 rows



qry: String = (select * from Nokia_Networks) as nokia
nkDf: org.apache.spark.sql.DataFrame = [Meas_Info_Id: string, Meas_Types: string ... 1 more field]
